# GDELT Pilot
### John Brandt

## Data Acquisition with BigQuery

In [ ]:
import pandas as pd

In [ ]:
from google.cloud import bigquery
client = bigquery.Client()

query = (
    
    "SELECT SourceCommonName, Amounts, V2Locations, V2Organizations, V2Themes FROM [gdelt-bq:gdeltv2.gkg@-604800000-] "
    'WHERE Amounts LIKE "%trees%"'
    'AND Amounts LIKE "%planted%"'
)
query_job = client.query(
    query,
    location="US",
)

for row in query_job:  # API request - fetches results
    # Row values can be accessed by field name or index
    assert row[0] == row.name == row["name"]
    print(row)

## Load in data

Because each BigQuery call costs about \$0.25 USD, I will load in a CSV during each jupyter session. This notebook currently works with weekly references to tree plantings, but will be functionalized to work with other event detections in the future.

In [16]:
files = os.listdir("../data/external")
data = pd.read_csv("../data/external/" + files[3])

## Locating events

In order to tie events to locations and organizations, we assign the location most closely referenced to the event in the text. 

For each event detected, create a dictionary of the form {index : (number, action)}. This will be matched with a similar location dictionary of the form {index : location} with a grid search.

In [77]:
for i, val in enumerate(data.iloc[0:10, 2]):               # Loop through each entry
    refs = ([x for x in val.split(";") if "trees" in x])   # Split up the references into value, action, index
    values, actions, indexes = [], [], []
    for ref in refs:                                       # Parse into separate lists
        parsed = ref.split(",")
        values.append(int(parsed[0]))
        actions.append(parsed[1])
        indexes.append(int(parsed[2]))
    refs = dict(zip(indexes, zip(values, actions)))        # {index: (number, action)}
    print(refs)

{507: (16, 'trees planted for more')}
{657: (1000000000, 'trees'), 1376: (300, 'trees between ash'), 2906: (300, 'trees planted for every')}
{2181: (21, 'willow trees implanted upside')}
{1068: (10000000, 'trees till date'), 1114: (22000000, 'trees planted by the')}
{729: (14000000, 'trees would be planted')}
{77: (60, 'volunteers planted 26 trees'), 96: (26, 'trees')}
{54: (2, 'mature trees'), 1889: (80, 'trees planted')}
{3: (250, 'native ghaf trees planted'), 87: (250, 'native ghaf trees planted'), 175: (50, 'ghaf trees were each')}
{69: (43, 'trees'), 437: (43, 'trees have been replanted')}
{69: (43, 'trees'), 437: (43, 'trees have been replanted')}


In [76]:
# TODO: Create a matching dictionary
for i in data.iloc[1:2, 3]:
    print(str(i).split(";"))

['1#Spanish#SP#SP##40#-4#SP#2488', '1#Spanish#SP#SP##40#-4#SP#4450', '1#India#IN#IN##20#77#IN#291', '1#India#IN#IN##20#77#IN#3916', '1#India#IN#IN##20#77#IN#4377', '4#Madrid, Madrid, Spain#SP#SP29#25820#40.4#-3.68333#-390625#677', '4#Madrid, Madrid, Spain#SP#SP29#25820#40.4#-3.68333#-390625#1613', '4#Mahadev, Jammu And Kashmir, India#IN#IN12#72851#32.8889#74.9014#6205823#3909', '4#Kathmandu, Bagmati, Nepal#NP#NP01#22353#27.7167#85.3167#-1022136#2796', '4#Kathmandu, Bagmati, Nepal#NP#NP01#22353#27.7167#85.3167#-1022136#3752', '1#Nepal#NP#NP##28#84#NP#299', '1#Nepal#NP#NP##28#84#NP#2289', '1#Nepal#NP#NP##28#84#NP#2714', '1#Nepal#NP#NP##28#84#NP#2803', '1#Nepal#NP#NP##28#84#NP#2983', '1#Nepal#NP#NP##28#84#NP#3759', '1#Nepal#NP#NP##28#84#NP#3850', '1#Nepal#NP#NP##28#84#NP#3933', '1#Nepal#NP#NP##28#84#NP#4385']
